# LoraQween: Colab Training

This notebook mounts Google Drive, sets up the project, installs deps, audits the dataset, and launches training for a person (e.g., Mikassa).


In [ ]:
# Check GPU availability
import torch, platform
print('Torch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU name:', torch.cuda.get_device_name(0))
print('Python:', platform.python_version())


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR_DRIVE = '/content/drive/MyDrive/LoraQween'
PROJECT_DIR = '/content/LoraQween'

print('Drive project:', PROJECT_DIR_DRIVE)
print('Local project:', PROJECT_DIR)


In [ ]:
# Copy project (code and dataset) from Drive to fast local storage
mkdir -p /content/LoraQween

# Core project
cp -r "$PROJECT_DIR_DRIVE/config.yaml" /content/LoraQween/ 2>/dev/null || true
cp -r "$PROJECT_DIR_DRIVE/scripts" /content/LoraQween/ 2>/dev/null || true
cp -r "$PROJECT_DIR_DRIVE/src" /content/LoraQween/ 2>/dev/null || true

# Dataset (person folder)
mkdir -p /content/LoraQween/data/datasets
cp -r "$PROJECT_DIR_DRIVE/data/datasets/Mikassa" /content/LoraQween/data/datasets/ 2>/dev/null || true

# Optional docs
cp -r "$PROJECT_DIR_DRIVE/docs" /content/LoraQween/ 2>/dev/null || true

ls -la /content/LoraQween


In [ ]:
# Install dependencies (excluding torch to keep Colab's preinstalled CUDA build)
import sys, subprocess

def pip_install(args):
    cmd = [sys.executable, "-m", "pip", "install", "-q", "-U"] + args
    print("Installing:", " ".join(args))
    subprocess.check_call(cmd)

pip_install(["pip", "setuptools", "wheel"])
pip_install([
    "transformers>=4.40.0",
    "diffusers>=0.29.0",
    "accelerate>=0.29.0",
    "peft>=0.10.0",
    "Pillow",
    "opencv-python-headless",
    "numpy",
    "pandas",
    "pyyaml",
    "tqdm",
    "psutil",
    "tensorboard",
    "exifread",
    "piexif",
    "pynvml",
])

import torch
print('Torch:', torch.__version__, '| CUDA available:', torch.cuda.is_available())


In [ ]:
# Env: set Hugging Face cache locally (fast), and persist outputs/loras to Drive via symlinks
import os, pathlib, subprocess

HF_CACHE = "/content/hf_cache"
pathlib.Path(HF_CACHE).mkdir(parents=True, exist_ok=True)
os.environ["HF_HOME"] = HF_CACHE
os.environ["HF_HUB_CACHE"] = str(pathlib.Path(HF_CACHE) / "hub")
print('HF_HOME =', os.environ["HF_HOME"])  
print('HF_HUB_CACHE =', os.environ["HF_HUB_CACHE"])  

# Link outputs and loras to Drive (to persist results)
DRIVE_BASE = "/content/drive/MyDrive/LoraQween"
LOCAL_BASE = "/content/LoraQween"

for rel in ["models/loras", "outputs", "logs"]:
    drive_path = pathlib.Path(DRIVE_BASE) / rel
    local_path = pathlib.Path(LOCAL_BASE) / rel
    drive_path.mkdir(parents=True, exist_ok=True)
    if local_path.exists() or local_path.is_symlink():
        subprocess.run(["rm", "-rf", str(local_path)], check=False)
    subprocess.run(["ln", "-s", str(drive_path), str(local_path)], check=False)
    print("Linked:", local_path, "->", drive_path)


In [ ]:
# Audit dataset to ensure integrity
import subprocess, os
os.chdir('/content/LoraQween')
subprocess.check_call(["python", "scripts/audit_dataset.py", "--person", "Mikassa"])


In [ ]:
# Launch training
import subprocess, os
os.chdir('/content/LoraQween')
subprocess.check_call(["python", "scripts/train_lora.py", "--person", "Mikassa"])
